In [0]:
#To install PyTorch-GPU on the system


from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' #'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'
print('Platform:', platform, 'Accelerator:', accelerator)

!pip install --upgrade --force-reinstall -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision

import torch
print('Torch', torch.__version__, 'CUDA', torch.version.cuda)
print('Device:', torch.device('cuda:0'))

In [0]:
#Import Dependencies

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import itertools

In [0]:
#The model class.
#Adversarial Training involves returning logits. adv_expt parameter in forward() does that.

class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.conv1=nn.Conv2d(kernel_size=3, stride=1, padding=1, in_channels=1, out_channels=32)
    self.bn1=nn.BatchNorm2d(32)
    self.relu1=nn.ReLU()
    self.maxpool1=nn.MaxPool2d(kernel_size=2, stride=2)
    self.conv2=nn.Conv2d(kernel_size=3, stride=1, padding=1, in_channels=32, out_channels=32)
    self.bn2=nn.BatchNorm2d(32)    
    self.relu2=nn.ReLU()
    self.maxpool2=nn.MaxPool2d(kernel_size=2, stride=2)
    self.fc1=nn.Linear(in_features=32*49, out_features=500)
    self.relu3=nn.ReLU()
    self.fc2=nn.Linear(in_features=500, out_features=10)
    self.softmax=nn.Softmax()

  def forward(self, input_, bn=False, adv_expt=False):
    out=self.conv1(input_)
    if(bn):
      out=self.bn1(out)
    out=self.relu1(out)
    out=self.maxpool1(out)
    out=self.conv2(out)
    if(bn):
      out=self.bn2(out)
    out=self.relu2(out)
    out=self.maxpool2(out)
    out=out.reshape(out.size(0),-1)
    out=self.fc1(out)
    out=self.relu3(out)
    logits=self.fc2(out)
    softmax=self.softmax(logits)
    if(adv_expt):
      return logits
    else:
      return softmax
    

In [0]:
def plot_loss(train_loss, test_loss, accuracy, steps=200):
    x=int(steps)*np.arange(0, len(train_loss))
    plt.plot(x,train_loss, label='Train')
    plt.plot(x,test_loss, label='Test')
    plt.title('Training Loss Plot')
    plt.xlabel("Iterations")
    plt.ylabel("Average Loss")
    plt.legend(loc='upper left')
    plt.show()
  
    plt.plot(x, accuracy, label='Accuracy')
    plt.title('Model Test Accuracy')
    plt.xlabel("Iterations")
    plt.ylabel("Average Loss")
    plt.show()   

    
def plot_images(show_images, show_labels, model, bn):
    pred=model(show_images, bn=bn)
    predlabels=torch.max(pred,1)[1].cpu().numpy()
    fig=plt.figure(figsize=(10, 10))
    columns = 3
    rows = 1
    for i in range(1, columns*rows +1):
        fig.add_subplot(rows, columns, i) 
        trueval=show_labels[i-1].data.cpu().numpy()
        predval=predlabels[i-1]
        a='True: '+str(trueval)+' Pred: '+str(predval)
        plt.title(a)
        plt.axis('off')
        plt.imshow(show_images[i-1].reshape(-1,28), cmap='gray')
    plt.show()


def visualise_weights(weights):
  fig=plt.figure(figsize=(10, 6))
  columns = 8
  rows = 4    
  for i in range(1, columns*rows +1):
      fig.add_subplot(rows, columns, i)
      plt.axis('off')
      plt.imshow(255*weights[i-1][0], cmap='gray')
  plt.colorbar()
  plt.show()


def train(model, epochs, bn=False):
  train_loss=[]
  test_loss=[]
  accuracy=[]
  for epoch in range(epochs):
    for i,(images, labels) in enumerate(trainloader):
      images=images.reshape(-1, 1, 28, 28).cuda()
      labels=labels.cuda()
      pred=model(images, bn=bn)
      train_avg_loss=criterion(pred, labels)
      optimizer.zero_grad()
      train_avg_loss.backward()
      optimizer.step()
      if(i%200==0):
        with torch.no_grad():
          correct = 0
          total = 0
          for test_images, test_labels in testloader:
            test_images=test_images.cuda()
            test_labels=test_labels.cuda()
            outputs = model(test_images, bn=bn)
            _, predicted = torch.max(outputs.data, 1)
            total += test_labels.size(0)
            correct += (predicted == test_labels).sum().item()
          test_avg_loss=criterion(outputs, test_labels)
          test_loss.append(test_avg_loss)  
          train_loss.append(train_avg_loss)
          acc=1.0*correct/total
          accuracy.append(acc)
          print("\nAccuracy ", acc, test_avg_loss.item())
  plot_loss(train_loss, test_loss, accuracy, 200)
  for test_images, test_labels in testloader:
      show_images=test_images[:3]
      show_images=show_images.cuda()
      show_labels=test_labels[:3]
      show_labels=show_labels.cuda()
      plot_images(show_images, show_labels, model, bn=bn)

      
def occlusion_prob(model, image, true_label):
  occ_size=12
  stride=4
  img_size=28
  occ_num=(img_size-occ_size)/stride+1
  zeros=torch.zeros((1,1,occ_size,occ_size), device='cuda')
  prob=np.zeros((occ_num,occ_num))
  for i,j in itertools.product(range(occ_num),range(occ_num)):
    a=np.arange(4*i,4*i+12,1)
    b=np.arange(4*j,4*j+12,1)
    for l,m in itertools.product(a,b):
      image[0][0][l][m]=0
    image=image.cuda()
    pred=model(image)
    prob[i][j]=pred[0][true_label] 
  return prob


def get_accuracy(model, bn=False):
  with torch.no_grad():
          correct = 0
          total = 0
          for test_images, test_labels in testloader:
            test_images=test_images.cuda()
            test_labels=test_labels.cuda()
            outputs = model(test_images, bn)
            _, predicted = torch.max(outputs.data, 1)
            total += test_labels.size(0)
            correct += (predicted == test_labels).sum().item()
          test_avg_loss=criterion(outputs, test_labels)
          acc=1.0*correct/total
          print("\nAccuracy ", acc, test_avg_loss.item())

          
def image_opt(model_):  
  fig=plt.figure(figsize=(12, 6))
  columns = 5
  rows = 2
  cost=[]
  for j in range(10):
    img_=np.zeros((1,1,28,28))+0.5
    img_=img_.astype(np.float32)
    img_=torch.from_numpy(img_)
    img_=torch.tensor(img_.type(torch.cuda.FloatTensor), requires_grad=True, device='cuda')
    zeros=torch.zeros((1,1,28,28), requires_grad=False, device='cuda')

    l=0.1
    fig.add_subplot(rows, columns, j+1)
    optimizer=torch.optim.SGD([img_], lr=0.05)
    for i in range(200):      
      optimizer.zero_grad()
      logits=model_(img_, adv_expt=True).cuda()
      pred=model_(img_, adv_expt=False).cuda()
      label=torch.tensor([j], device='cuda')
      loss2=logits[0][j]
      crit3=nn.MSELoss(size_average=False)
      loss3=l*crit3(img_, zeros)
      loss=-loss2+loss3
      loss.backward()
      optimizer.step()
      if(j==1):
        cost.append(loss.item())

    plt.axis('off')
    plt.title(str(j))
    plt.imshow(img_.detach().cpu().numpy().reshape(-1,28), cmap='gray')
  plt.show()
  fig=plt.figure()
  x=5*np.arange(0, len(cost))
  plt.plot(x, cost)
  plt.title('Cost')
  plt.xlabel("Iterations")
  plt.show()

In [0]:
train_batch_size=64
test_batch_size=10000
lr=0.00005

In [0]:
#Load data - 50k train and 10k test split
transform=transforms.ToTensor()
traindata=torchvision.datasets.MNIST(root='drive_path', download=True, train=True, transform=transform)
testdata=torchvision.datasets.MNIST(root='drive_path', download=True, train=False, transform=transform)
trainloader=torch.utils.data.DataLoader(traindata, batch_size=train_batch_size, shuffle=True)
testloader=torch.utils.data.DataLoader(testdata, batch_size=test_batch_size)

In [0]:
#Train Model 1

criterion=nn.CrossEntropyLoss().cuda()
model=CNN().cuda()
optimizer=torch.optim.Adam(model.parameters(), lr=lr)
train(model, 10)
get_accuracy(model)

In [0]:
#Train Model 2

criterion=nn.CrossEntropyLoss().cuda()
modelbn=CNN().cuda()
optimizer=torch.optim.Adam(modelbn.parameters(), lr=lr)
train(modelbn, 10, bn=True)
get_accuracy(modelbn, bn=True)

In [0]:
#Freeze model parameters
model_=model
for child in model_.children():
  for param in child.parameters():
    param.requires_grad=False

In [0]:
#Visualize model parameters
conv1=list(model_.children())[0]
conv2=list(model_.children())[4]
conv1_weights=(list(conv1.parameters())[0]).data.cpu().numpy()
conv2_weights=(list(conv2.parameters())[0]).data.cpu().numpy()
visualise_weights(conv1_weights)
visualise_weights(conv2_weights)

In [0]:
#Perform the occlusion experiment

for images,labels in testloader:
  image=images[:10]
  label=labels[:10]
  fig=plt.figure(figsize=(12, 6))
  columns = 5
  rows = 2
  for i in range(1, columns*rows +1):
        fig.add_subplot(rows, columns, i)
        om=occlusion_prob(model_, image[i-1][0].reshape(-1,1,28,28), label[i-1])
        trueval=labels[i-1].data.cpu().numpy()
        plt.title(trueval)
        plt.imshow(om, cmap='gray')
        plt.grid('off')   
  plt.colorbar()
  plt.show()

In [0]:
trainloader_=torch.utils.data.DataLoader(traindata, batch_size=1, shuffle=True)
img1, label1=iter(trainloader_).next()
img2, label2=iter(trainloader_).next()

In [0]:
#Performs the non-targeted attack experiment

adv=np.random.normal(loc=0.5, scale=0.05, size=(1,1,28,28))
adv=adv.astype(np.float32)
adv=torch.from_numpy(adv)
adv = torch.tensor(adv.type(torch.cuda.FloatTensor), requires_grad=True, device='cuda')
cost=[]
optimizer=torch.optim.Adam([adv], lr=0.0005)
print(model_(adv, adv_expt=True))
for i in range(200):
  optimizer.zero_grad()
  logits=model_(adv, adv_expt=True)
  loss=-logits[0][5]
  loss.backward()
  optimizer.step()
  if(i%5==0):
    cost.append(loss.item())
    
print(model_(adv, adv_expt=True))

img=adv
img=img.cpu().detach().numpy()
plt.grid('off')
plt.axis('off')
plt.imshow(img[0][0], cmap='gray')
plt.colorbar()
x=5*np.arange(0, len(cost))
plt.plot(x, cost)
plt.title('Cost')
plt.xlabel("Iterations")
plt.show()


In [0]:
#Performs the targeted-attack experiment

adv=np.random.normal(loc=0.5, scale=0.05, size=(1,1,28,28))
adv=adv.astype(np.float32)
adv=torch.from_numpy(adv1)
adv=torch.tensor(adv.type(torch.cuda.FloatTensor), requires_grad=True, device='cuda')

b=1
optimizer=torch.optim.SGD([adv], lr=0.001)
print(model_(adv, adv_expt=True))
for i in range(400):
  optimizer.zero_grad()
  logits=model_(adv, adv_expt=True)
  loss1=-logits[0][8]
  criterion=nn.MSELoss()
  img1=img1.cuda()
  loss2=criterion(adv, img1)
  loss=loss1+b*loss2
  loss.backward()
  optimizer.step()
    
print(model_(adv, adv_expt=True))
img=adv
img=img.cpu().detach().numpy()
plt.imshow(img[0][0], cmap='gray')
plt.colorbar()
plt.grid('off')
plt.axis('off')

In [0]:
#Performs the additive noise experiment


#For class 4
noise1=torch.zeros((1,1,28,28), requires_grad=True, device='cuda')
img1=img1.cuda()
optimizer=torch.optim.SGD([noise1], lr=0.002)
print(model_(noise1+img1, adv_expt=True))
for i in range(400):
  optimizer.zero_grad()
  logits=model_(noise1+img1, adv_expt=True)
  loss=-logits[0][5]
  loss.backward()
  optimizer.step()

noise=noise1
print(model_(noise1+img1, adv_expt=True))
img=noise1+img1
img=img.cpu().detach().numpy()
noise=noise.cpu().detach().numpy()

fig=plt.figure(figsize=(10, 5))  
fig.add_subplot(1,2,1)
plt.imshow(img[0][0], cmap='gray')
plt.grid('off')
plt.axis('off')
fig.add_subplot(1,2,2)
plt.imshow(noise[0][0], cmap='gray')
plt.grid('off')
plt.axis('off')

#For the class 5
noise2=torch.zeros((1,1,28,28), requires_grad=True, device='cuda')
img1=img1.cuda()

optimizer=torch.optim.SGD([noise2], lr=0.002)
print(model_(noise2+img1, adv_expt=True))
for i in range(400):
  optimizer.zero_grad()
  logits=model_(noise2+img1, adv_expt=True)
  loss=-logits[0][4]
  loss.backward()
  optimizer.step()

noise=noise2
print(model_(noise2+img2, adv_expt=True))
img=noise2+img1
img=img.cpu().detach().numpy()
noise=noise.cpu().detach().numpy()

fig=plt.figure(figsize=(10, 5))  
fig.add_subplot(1,2,1)
plt.imshow(img[0][0], cmap='gray')
plt.grid('off')
plt.axis('off')
fig.add_subplot(1,2,2)
plt.imshow(noise[0][0], cmap='gray')
plt.grid('off')
plt.axis('off')
    

In [0]:
#For class 4
for images, labels in testloader:
  images=images[:10]
  labels=labels[:10]
  noise1=noise1.cuda()
  images=images.cuda()
  images=images+noise1
  pred=model_(images)
  predlabels=torch.max(pred,1)[1].cpu().numpy()
  fig=plt.figure(figsize=(12, 6))
  columns = 5
  rows = 2
  for i in range(1, columns*rows +1):
      fig.add_subplot(rows, columns, i) 
      trueval=labels[i-1].data.cpu().numpy()
      predval=predlabels[i-1]
      a='True: '+str(trueval)+' Pred: '+str(predval)
      plt.title(a)
      plt.axis('off')
      plt.imshow(images[i-1].reshape(-1,28).cpu().detach().numpy(), cmap='gray')
  plt.show()

#For class 5
for images, labels in testloader:
  images=images[:10]
  labels=labels[:10]
  noise2=noise2.cuda()
  images=images.cuda()
  images=images+noise2
  pred=model_(images)
  predlabels=torch.max(pred,1)[1].cpu().numpy()
  fig=plt.figure(figsize=(12, 6))
  columns = 5
  rows = 2
  for i in range(1, columns*rows +1):
      fig.add_subplot(rows, columns, i) 
      trueval=labels[i-1].data.cpu().numpy()
      predval=predlabels[i-1]
      a='True: '+str(trueval)+' Pred: '+str(predval)
      plt.title(a)
      plt.axis('off')
      plt.imshow(images[i-1].reshape(-1,28).cpu().detach().numpy(), cmap='gray')
  plt.show()

In [0]:
#Performs the optimization over images experiment
image_opt(model_)